In [14]:
import re
import csv
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from unicodedata import normalize
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

The function of the next cell clicks the correct tab within the website, introduces the surname on the search box and click search, switches to the new tab and gets the html from that tab and read all tables and chooses a  particular table that contains the data for the freqency of ocurrence of the surname.

In [58]:
def Surname_Data(Name):
    from selenium.webdriver.common.keys import Keys
    
    options = Options()
    #options.add_argument("--headless") # Runs Chrome in headless mode.
    #options.add_argument('--no-sandbox') # # Bypass OS security model
    options.add_argument('start-maximized')
    options.add_argument('disable-infobars')
    options.add_argument("--disable-extensions")
    
    driver = webdriver.Chrome(chrome_options=options,executable_path='/home/erq/chromedriver')
    url = 'https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177009&menu=resultados&secc=1254736195497&idp=1254734710990#!tabs-1254736195497'
    driver.get(url)       

    time.sleep(1)
    
    # introduce the surname on the search box
    surname = driver.find_elements_by_id('cmb6')[1]
    surname.clear()
    surname.send_keys(Name)

    # click search
    driver.find_element_by_xpath('//*[@id="tapellidos1"]/p[2]/button[2]').click()

    time.sleep(1)
    
    # change tab, this next loop helps visualize what the browser is doing
    handles = []
    for handle in driver.window_handles:
        handles.append(handle)
        print(handle)
    
    time.sleep(1)

    # Choose the second window handle = change tab
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)
    
    # Get html
    html_from_page = driver.page_source
    
    driver.quit() 
    
    tables = pd.read_html(html_from_page)
    
    Tabla  = tables[4]
    return(Tabla)

In [57]:
Surname_Data('Barrueco')

/home/erq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


CDwindow-34A80157399DC1A53018E9CD9FE634B5
CDwindow-99B5462C7D1BF6503EA0DC76A3E952AD


Provincia Apellido 1º             Apellido 2º              \
                  Provincia       Total Por mil (‰)       Total Por mil (‰)   
0                     Total         571        0012         603          13   
1          Alicante/Alacant          ..          ..           6           5   
2               Araba/Álava          ..          ..           8          35   
3                   Badajoz          ..          ..          14          16   
4            Balears, Illes           9        0013          10          15   
5                 Barcelona          39        0010          37           9   
6                   Bizkaia          45        0045          32          32   
7                   Cáceres          18        0032          19          33   
8                     Cádiz          14        0011           8           6   
9                 Cantabria           6        0011           5           9   
10              Ciudad Real          11        0017           6           9   
11                 Gipuzkoa           5        0008          10          16   
12                   Madrid         109        0025         109          25   
13                   Murcia          ..          ..          12           9   
14                Salamanca         147        0339         184         425   
15   Santa Cruz de Tenerife          ..          ..           6           8   
16                  Sevilla          10        0005          11           6   
17                   Toledo          18        0029           8          13   
18        Valencia/València          10        0005          23          12   
19               Valladolid          ..          ..           9          18   
20                   Zamora          75        0274          42         153   
21  Nacido en el extranjero          26        0004          21           3   

   Ambos apellidos              
             Total Por mil (‰)  
0               20          ..  
1               ..          ..  
2               ..          ..  
3               ..          ..  
4               ..          ..  
5               ..          ..  
6               ..          ..  
7               ..          ..  
8               ..          ..  
9               ..          ..  
10              ..          ..  
11              ..          ..  
12              ..          ..  
13              ..          ..  
14              17        0039  
15              ..          ..  
16              ..          ..  
17              ..          ..  
18              ..          ..  
19              ..          ..  
20              ..          ..  
21              ..          ..

This next function looks for the origin of a surname in the website www.misapellidos.com when a surname is 
fed to it. It can run headless or not. It opens the website for that surname, gets the html, turns it into soup 
and finds the surnames shown on the website for that query, then it matches the surname given to the function and 
returns the origin of the surname if there is a match. If no match is found, then it returns 0.

In [61]:
def Get_Origin(Name):
    
    options = Options()
    options.add_argument('start-maximized')
    
    # uncomment these next 2 lines if you want to go headless, you won't see the browser is doing. 
    options.add_argument("--headless") # Runs Chrome in headless mode.
    options.add_argument('--no-sandbox') # # Bypass OS security model
    
    print ("Headless Chrome Initialized on Linux OS")

    #Change executable path of chromedriver.
    driver = webdriver.Chrome(chrome_options=options,executable_path='/home/erq/chromedriver')
    
    url = f'https://www.misapellidos.com/buscar.php?busqueda=combinada&texto={Name}&buscar=Buscar'
    driver.get(url)       

    time.sleep(1)
    
    html_from_page = driver.page_source
    soup = BeautifulSoup(html_from_page, 'html.parser')
    driver.close() 

    apellidos_encontrados = (soup.find_all('td', attrs ={'width':'565'}))
    Origin = ""
    for i in range(len(apellidos_encontrados)):
        match = (soup.find_all('td', attrs ={'width':'565'})[i].text.split(' Breve Descripción de ')[1])
        
        # This gets rid of accents.
        match = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", match), 0, re.I)
        match = normalize( 'NFC', match)

        print('Match:', match)
        if match == Name.capitalize():
    
            Origin = soup.find_all('td', attrs ={'width':'85'})[i].find_next('img').find_next('img').get('src').split('/')[-1].split('.')[0].capitalize()
            print(match, '----->', Origin)
            break
        elif match.capitalize() == Name.capitalize():
            Origin = soup.find_all('td', attrs ={'width':'85'})[i].find_next('img').find_next('img').get('src').split('/')[-1].split('.')[0].capitalize()
            print(match, '----->', Origin)
            break         

        else:
            pass
            print('Not the first one')
    if Origin == "":
        print('Origin not found for', Name)
        Origin = 0
        
    return(Origin)

    

In [62]:
Get_Origin('Barrueco')

Headless Chrome Initialized on Linux OS


/home/erq/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


Match: barrueco
barrueco -----> Spain


'Spain'

This next cell is part of the code for one of the programs I wrote from scratch for the company I work for. Even though all of the websites are quite similar each of them has a different structure and therefore the relevant information wont be found on the same place. Each of the websites have a function constructed to obtain the relevant information when a url is given to them.

In [ ]:
def find_function(url):

    if 'theguardian' in url:
        Results = Guardian(url)
        
    elif 'standard' in url:
        Results = Standard(url)
        
    elif 'telegraph' in url:
        Results = Telegraph(url)
        
    elif 'independent' in url:
        Results = Independent(url)
        
    elif 'elpais' in url:
        
        Results = El_Pais(url)
    
    elif 'elmundo' in url:
        Results = El_Mundo(url)
    
    elif 'elcultural' in url:
        Results = El_Cultural(url)
        
    elif 'elconfidencial' in url:
        Results = El_Confidencial(url)
    
    elif 'masdearte' in url:
        Results = MasDeArte(url)
    
    elif 'revistadearte' in url:
        Results = RevistaDeArte(url)
        
    elif 'timeout' in url:
        Results = TimeOut(url)
        
    elif 'artlyst' in url:
        Results = Artylst(url)
        
    elif 'theartsdesk' in url:
        Results = TheArtDesk(url)
        
    elif 'artreview' in url:
        Results = ArtReview(url)
        
    elif 'theartnewspaper' in url:
        Results = TheArtNewsPaper(url)
        
    elif 'spectator' in url:
        Results = Spectator(url)
        
    elif 'scotsman' in url:
        Results = Scotsman(url)
        
    elif 'theinfatuation' in url:
        Results = TheInfatuation(url)
        
    elif 'theweek' in url:
        Results = TheWeek(url)
        
    elif 'birminghammail' in url:
        Results = BirminghamMail(url)
        
    elif 'andyhayler' in url:
        Results = AndyHayler(url)
        
    elif 'gq-magazine' in url:
        Results = GQMagazine(url)
        
    elif 'walesonline' in url:
        Results = WalesOnline(url)
        
    elif 'pickyglutton' in url:
        Results = PickyGlutton(url)
        
    elif 'thecourier' in url:
        Results = TheCourier(url)
        
    elif 'squaremeal' in url:
        Results = SquareMeal(url)
        
    elif 'yorkshirepost' in url:
        Results = YorkShireReport(url)
        
    elif 'edp24' in url:
        Results = edp24(url)
        
    elif 'manchestereveningnews' in url:
        Results = ManchesterEveningNews(url)
        
    elif '5pm' in url:
        Results = five_pm(url)
        
    elif 'esquire' in url:
        Results = Esquire(url)
        
    elif 'bristolpost' in url:
        Results = BristolPost(url)
        
    elif 'devonlive' in url:
        Results = DevonLive(url)
        
    elif 'restaurantsbrighton' in url:
        Results = RestaurantsBrighton(url)
        
    elif 'linternaute' in url:
        Results = Linterature(url)
        
    elif 'gaultmillau' in url:
        Results = GaultMillau(url)
        
    elif 'offi' in url:
        Results = Offi(url)
        
    elif 'lefigaro' in url:
        Results = LeFigaro(url)
        
    elif 'liberation' in url:
        Results = Liberation(url)
        
    elif 'vsd' in url:
        Results = Vsd(url)
        
    elif 'lepoint' in url:
        Results = LePoint(url)
        
    elif 'lesechos' in url:
        Results = Lesechos(url)
        
    elif 'la-croix' in url:
        Results = LaCroix(url)
        
    elif 'lexpress' in url:
        Results = LeExpress(url)
        
    elif 'midilibre' in url:
        Results = MidiLibre(url)
        
    elif 'dna' in url:
        Results = DNA(url)
        
    elif 'leprogres' in url:
        Results = LeProgress(url)
        
    elif 'elle' in url:
        Results = Elle(url)
        
    elif 'lamontagne' in url:
        Results = LaMontage(url)
        
    elif 'alfayomega' in url:
        Results = AlfayOmega(url)
        
    elif 'eldiario' in url:
        Results = ElDiario(url)
        
    elif 'corriere' in url:
        Results = Corriere(url)
        
    elif 'repubblica' in url:
        Results = Repubblica(url)
        
    elif 'lastampa' in url:
        Results = LaStampa(url)
        
    elif 'ilgiornaledellarte' in url:
        Results = IlGiornaleDellArte(url)
        
    elif 'arte.it' in url:
        Results = Arte_it(url)
        
    elif 'tg24' in url:
        Results = tg24(url)
        
    elif 'itinerarinellarte' in url:
        Results = Itinerarinellarte(url)
        
    elif 'artslife' in url:
        Results = ArtsLife(url)
        
    elif 'museionline' in url:
        Results = Museionline(url)
        
    elif 'artemagazine' in url:
        Results = ArteMagazine(url)
        
    elif 'zingarate' in url:
        Results = Zingarate(url)
        
    elif 'gallerieditalia' in url:
        Results = GallerieDiItalia(url)
        
    elif 'inexhibit' in url:
        Results = Inexhibit(url)
        
    elif 'civita' in url:
        Results = Civita(url)
        
    elif 'dissapore' in url:
        Results = Dissapore(url)
        
    elif 'cucchiaio' in url:
        Results = Cucchiaio(url)
        
    elif 'oraviaggiando' in url:
        Results = Oraviaggiando(url)
        
    elif 'dimeunrestaurante' in url:
        Results = DimeUnRestaurante(url)
        
    elif 'gastroactitud' in url:
        Results = GastroActitud(url)
        
    elif 'elespanol' in url:
        Results = ElEspañol(url)
        
    elif 'complicidadgastronomica' in url:
        Results = ComplicidadGastronomica(url)
        
    elif 'unblogdepalo' in url:
        Results = UnBlogDePalo(url)
        
    elif 'mesade2' in url:
        Results = MesaDe2(url)
        
    elif 'gdegastronomia' in url:
        Results = GdeGastronomia(url)
        
    elif 'latimes' in url:
        Results = LATimes(url)
        
    elif 'nytimes' in url:
        Results = NYTimes(url)
    
    elif 'washingtonpost' in url:
        Results = WashingtonPost(url)
        
    elif 'inquirer' in url:
        Results = Inquirer(url)
        
    elif 'artforum' in url:
        Results = ArtForum(url)
        
    elif 'frieze' in url:
        Results = Frieze(url)
        
    elif 'hyperallergic' in url:
        Results = HyperAllergic(url)
        
    elif 'theartwolf' in url:
        Results = TheArtWolf(url)
        
    elif 'beyondshipsart' in url:
        Results = BeyondShipArt(url)
        
    elif 'thecultureconcept' in url:
        Results = TheCultureConcept(url)
        
    elif 'gayot' in url:
        Results = Gayot(url)
    
    elif 'vam.ac' in url:
        Results = VandA(url)
    
    else:
        print('ERROR, link not in sources')
        
        Results = ""
    return Results
    
    
def fetch_results(search_term, number_results, user_agent):
    #This function is used in the big for loop to scrape the data of each Place
    
    escaped_search_term = search_term.replace(' ', '+')
    google_url = 'https://www.google.com/search?q={}&num={}&hl=en-en'.format(escaped_search_term, number_results)
    response = requests.get(google_url, headers=user_agent)
    response.raise_for_status()
 
    return response.text
